# Clippy on Google Gemini!
### Mr Clippy will get it done!

#### Installing necessary packages and directory

In [1]:
# importing gemini api
import google.generativeai as genai
import os
import pandas as pd
import json
import time

# input api key payload
genai.configure(api_key = "YOUR API KEY HERE")

In [3]:
# Changing of working directory
os.chdir("YOUR FILE PATH HERE")

#### Setting prompts for Gemini

In [4]:
# Prompt  of model
model=genai.GenerativeModel(
    model_name="gemini-1.5-flash", 
    system_instruction='''You are Clippy, an AI research assistant. \
                    Evaluate provided research paper abstracts \
                    for relevance to a literature review on Large \
                    Language Models in patient self-care. Respond \
                        in JSON format with these fields:
                    {'relevance_yes_no': yes/no,
                     'relevance': One-sentence explanation of relevance,
                     'purpose_of_study': Brief description of research purpose}\
                    Be concise. If uncertain: 'relevance_yes_no': "Review full text", others: "Insufficient information"''')

#### Unleash Clippy!

In [ ]:
# File path
df = pd.read_csv("YOUR CSV FILE")
df

# Filling in blanks with NA text
df=df.fillna(value=('Missing'))
df

# Create an empty list to append
result=[]
batch_size = 10

for start_index in range(0, len(df), batch_size):
    end_index = min(start_index + batch_size, len(df))
    
    print(f"Processing rows {start_index} to {end_index-1}")
    
    
    for index, row in df.iloc[start_index:end_index].iterrows():
        abstract = row['Abstract']
        abstract_summary = model.generate_content(
            abstract, 
            generation_config=genai.GenerationConfig(
                response_mime_type='application/json'
            )
        )
        result.append(pd.json_normalize(json.loads(abstract_summary.text)))

    print(f"Completed processing batch {start_index//batch_size + 1}")
    time.sleep(60) # to prevent hitting rate limit

print("All batches processed.")
print(result)

In [ ]:
# Stick the columns together using concat
appended_data=pd.concat(result)
lit_review=pd.concat([df, appended_data.reset_index(drop=True)], axis=1)
lit_review

# Output to csv file
lit_review.to_excel('FILE_NAME.xlsx', index=False)